In [18]:
import plotly.express as px
from lostructpy_pkg import lostruct
import numpy as np
from skbio.stats.ordination import pcoa
from sklearn.manifold import MDS
import pandas as pd
import umap
import hdbscan
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
import lostructpy_pkg.lostruct as ls

In [3]:
windows, positions = ls.parse_vcf("chr1-filtered.vcf.gz", "chr1", 95)

In [4]:
result = list()
for x in windows:
    result.append(ls.eigen_windows(x, 10))
result = np.vstack(result)
pc_dists = ls.get_pc_dists(result)
mds = pcoa(pc_dists)

/home/josephguhlin/anaconda3/envs/gwasp/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning:

Casting complex values to real discards the imaginary part

/mnt/g/Development/lostruct-py/lostructpy_pkg/lostruct.py:111: ComplexWarning:

Casting complex values to real discards the imaginary part



In [5]:
mds_coords = pd.read_csv("lostruct-results/mds_coords.csv")
np.corrcoef(mds.samples['PC1'], mds_coords['MDS1'].to_numpy())

array([[1.        , 0.99785909],
       [0.99785909, 1.        ]])

In [6]:
px.scatter(x=mds.samples["PC1"], y=mds_coords['MDS1'])

In [7]:
px.scatter(y=mds.samples["PC1"])

In [8]:
px.scatter(y=mds.samples["PC2"])

# Some looks at other methods of clustering

In [29]:
embedding = MDS(n_components=10, dissimilarity="precomputed", n_jobs=-1, n_init=32)
mds = embedding.fit_transform(pc_dists)
px.scatter(y=[mds[:,0], mds_coords['MDS1']])

In [9]:
import phate
phater = phate.PHATE(n_components=10, knn_dist='precomputed', mds_solver='smacof', mds='metric')
comparison_phate = phater.fit_transform(pc_dists)

Calculating PHATE...
  Running PHATE on precomputed distance matrix with 124 observations.
  Calculating graph and diffusion operator...
    Calculating affinities...
  Calculating optimal t...
    Automatically selected t = 12
  Calculated optimal t in 0.03 seconds.
  Calculating diffusion potential...
  Calculating metric MDS...
  Calculated metric MDS in 0.45 seconds.
Calculated PHATE in 0.51 seconds.


In [10]:
px.scatter(y=[mds_coords['MDS1'], mds.samples["PC1"], comparison_phate[:,0]])

In [11]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(pc_dists)
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [12]:
hdbscan_labels = hdbscan.HDBSCAN().fit_predict(embedding)
px.scatter(x=embedding[:, 0], y=embedding[:, 1], color=hdbscan_labels)

In [13]:
reducer = umap.UMAP(n_components=3)
embedding = reducer.fit_transform(pc_dists)
hdbscan_labels = hdbscan.HDBSCAN().fit_predict(embedding)
fig = px.scatter_3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2], color=hdbscan_labels, width=800, height=600)
fig.show()